In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import random
import pandas as pd
from sklearn.metrics import davies_bouldin_score as DB

In [ ]:
class individuo:
  def __init__(self,k_max,k_min,x_min,x_max,d):
    self.cromosoma = list(np.random.random(k_max))
    self.checar(k_max,k_min)

    for j in range(k_max): #creamos los centroides
      self.cromosoma.append(list(np.random.uniform(low=x_min,high=x_max,size=d)))
  def checar(ind,k_max,k_min):
    ind.activos = [i for i in range(k_max) if ind.cromosoma[i] > 0.5]
    while len(ind.activos) < k_min:
      ind.cromosoma = list(np.random.random(k_max))
      ind.activos = [i for i in range(k_max) if ind.cromosoma[i] > 0.5]
  aptitud = None
  objetivo = None
  particion = None

In [ ]:
class individuo_vacio:
  cromosoma = []
  activos = []
  aptitud = None
  objetivo = None
  particion = None

In [ ]:
def euclideana(x1,x2,flag = False):
  '''la bandera nos indica si se calcula distancia entre datos 
  o entre un dato y un centroide, cuando es True indica distancia
  entre 2 puntos'''
  global distanciasP
  if flag == False:
    return( np.linalg.norm(np.array(x1)-np.array(x2) ) )
  else:
    try:
      return(distanciasP[str(x1+x2)])
    except:
      try:
        return(distanciasP[str(x2+x1)])
      except:
        dist = np.linalg.norm(np.array(x1)-np.array(x2) )
        distanciasP[str(x2+x1)] = dist
        distanciasP[str(x1+x2)] = dist
        return(distanciasP[str(x1+x2)])

In [ ]:
def coseno(x1 , x2, flag = False ):
  '''la bandera nos indica si se calcula distancia entre datos 
  o entre un dato y un centroide, cuando es True indica distancia
  entre 2 puntos'''
  global distanciasP
  if flag == False:
    return( dot(x1, x2)/(norm(x1)*norm(x2)) )
  else:
    try:
      return(distanciasP[str(x1+x2)])
    except:
      try:
        return(distanciasP[str(x2+x1)])
      except:
        dist = dot(x1, x2)/(norm(x1)*norm(x2))
        distanciasP[str(x2+x1)] = dist
        distanciasP[str(x1+x2)] = dist
        return(distanciasP[str(x1+x2)])

In [ ]:
def obtener_elementos(particion,cluster,datos):
  '''
  Devuelve los elemetos que forman parte del cluster especificado,
  ya que los guardamos en una lista simple, donde el indice indica 
  el numero del elemento y el valor indica el cluster al que pertenece.
  '''
  return( [datos[i] for i in range(len(particion)) if particion[i] == cluster])

In [ ]:
def repartir_datos(len_datos,activos):
  r = []
  c = 0
  while len(r) < len_datos:
    r.append(activos[c])
    if c !=len(activos)-1:
      c+=1
    else:
      c=0
  return(r)

In [ ]:
#version funcional
def completar_individuo(individuo , datos : list,k_max : int, f_distancia,x:min,x_max,aptitud,eps = 0.0001):
  '''esta funcion agrega los datos que le hacen falta a los individuos cuando
  son recien creados, le agrega su particion de elementos y el valor de que
  toma en la funcion objetivo '''

  'primero agregamos la particion'
  particion = [] 
  for dato in datos:
    #cambiar por un for
    centro = 0
    menor = f_distancia( dato,individuo.cromosoma[k_max] )
    for i in range(1,k_max):
      if individuo.cromosoma[i] > 0.5:
        x = f_distancia( dato,individuo.cromosoma[k_max+i] )
        if x < menor:
          menor = x
          centro = i
    particion.append(centro)
  individuo.particion = particion

  'Algunos cromosomas dan errores, aqui verificamos si es el caso, si asi fuera, lo solucionamos cambiando la particion'
  for grupo in individuo.activos:
    cant_ele = 0 # sum([1 for i in individuo.particion if i == grupo])
    for punto in individuo.particion:
      if punto == grupo:
        cant_ele +=1
    'un cromosoma defectuosos es el que contiene algun centro con menos de 2 elementos, verificamos si esto pasa'
    if cant_ele < 2:
      individuo.particion = repartir_datos(len(datos),individuo.activos)
      break
  individuo.objetivo = aptitud(individuo,datos,k_max,f_distancia)
  individuo.aptitud = 1 / (individuo.objetivo + eps)

In [ ]:
def calcular_centroide(centro,elementos):
  return(list(np.mean(elementos,axis = 0)))

In [ ]:
#version final sin listas comprension
def cs(individuo, datos : list,k_max,f_distancia):
  dividendo = []
  divisor = []
  centros = individuo.cromosoma[k_max::]
  for centro in individuo.activos:
    elementos = obtener_elementos(individuo.particion,centro,datos)
    centros[centro] = calcular_centroide(centro,elementos)
    'vamos a calcular la distancia mas grande de un punto a cualquier otro, para todos los puntos en el grupo'
    distancias_puntos = []
    for i in range(len( elementos)):
      max = 0
      for j in range(len(elementos)):
        if j != i:
          dist = f_distancia(elementos[i],elementos[j])
          if dist > max:
            max = dist
      distancias_puntos.append(max)
    dividendo.append(np.mean(distancias_puntos))
    'ahora calculamos las mimimas distancias entre los centroides de cada grupo'
    min = 9999999
    for x in individuo.activos:
      if x != centro:
        entre_grupos = f_distancia(centros[centro],centros[x])
        if entre_grupos < min:
          min = entre_grupos
    divisor.append( min  )
  'actualizamos la informacion del individuo, ya que alteramos una copia de sus datos'
  individuo.cromosoma[k_max::] = centros
  return(np.mean(dividendo) / np.mean(divisor))

In [ ]:
def imprimir(soluciones,m):
  mejor = soluciones[0]
  peor = soluciones[0]
  grupos = []
  puntuaciones = []
  for sol in soluciones:
    grupos.append(len(sol.activos))
    puntuaciones.append(sol.objetivo )
    if sol.objetivo < mejor.objetivo:
      mejor = sol
    if sol.objetivo > peor.objetivo:
      peor = sol
  grupos_p = np.mean(grupos)
  std = np.std(puntuaciones) ; media = np.mean(puntuaciones);
  print(f'DEspues de {m} iteraciones, estos son los resultados:')
  print(f'La mejor solucion fue {mejor.cromosoma}' )
  print(f'con un valor en la funcion objetivo de {mejor.objetivo}')
  print(f'La peor solucion fue {peor.cromosoma}' )
  print(f'con un valor en la funcion objetivo de {peor.objetivo}')
  print(f'En promedio se obtiene un valor en F(x) de {media}')
  print(f'Se tiene una desviacion estandar de {std}')
  print(f'Las soluciones tienen en promedio {grupos_p} grupos')

In [ ]:
def ACDE(cr_max, cr_min, k_max, k_min, pop_size, x_min, x_max, d : int, max_eval, datos : list,f_distancia,aptitud,M : int,eps = 0.0001):
  '''
  cr_max es el valor maximo que puede tomar la constante de cruza
  cr_min es el minimo valor que puede tomar la CR
  k_max el numero maximo de grupos que puede haber
  k_min el minimo de rupos que deben de considerarse
  pop_size tamaño de la poblacion
  x_max y x_min sirven para delimitar el rango de valores que pueden
  tener los datos, por ejemplo, estan en el rango (0,2)
  d es la dimension de los datos R2 : (x,y), R3 : (x,y,z) etc
  max_eval es la cantidad maxima de llamadas a la funcion aptitud CS
  datos es una lista con los datos a ordenar
  ADVERTENCIA LA CANTIDAD DE DATOS DEBE SER AL MENOS 2 VECES LA CANTIDAD DE GRUPOS
  '''
  global distanciasP
  distanciasP= {}
  mejores = []
  for iterador in range(M):
    'creamos la poblacion inicial y le agregamos sus particiones y valores en CS'
    poblacion = [individuo(k_max,k_min,x_min,x_max,d) for i in range(pop_size)]
    for ind in poblacion:
      completar_individuo(ind, datos, k_max,f_distancia,x_min,x_max,aptitud,eps)
    'iteramos el algoritmo t generaciones, para saber cuantas deben ser, al numero de evaluaciones restamos pop_size y dividimos entre pop_size'
    iter = (max_eval)//pop_size
    best = sorted(poblacion, key= lambda x: x.aptitud, reverse=True)[0]

    'la siguiente lista es solo para darse una idea de cuanto va avanzando el algoritmo'
    avance = np.linspace(0,10000,num=100)

    for t in range(1,iter+1):
      cr = (cr_max-cr_min) * ((iter-t) / iter)
      F = 0.5*(1+np.random.random())
      'recorremos individuo a individuo para aplicar ED'
      for ind in poblacion:
        intento = individuo_vacio()
        a,b,c = random.sample(poblacion,3)
        'hacemos la cruza y mutacion para darle valores al individuo intento'
        l = []
        for i in range(len(ind.cromosoma)):
          if np.random.random() < cr:
            if i < k_max:
              res = c.cromosoma[i] + F*(a.cromosoma[i]-b.cromosoma[i])
              if res >= x_min and res <= x_max: #verificamos que el resultado de la cruza no se salga del intervalo de busqueda
                l.append(res)
              else:
                l.append( np.random.uniform(low=x_min,high=x_max) )
            else:
              cruzado = [c.cromosoma[i][j] + F*(a.cromosoma[i][j]-b.cromosoma[i][j]) for j in range(d)     ]
              for x in range(d): #vemos que los centros no se salgan del espacio de busqueda
                if cruzado[x] < x_min or cruzado[x] > x_max:
                  cruzado[x] = np.random.uniform(low=x_min,high=x_max)
              l.append( cruzado  )
          else:
            l.append(ind.cromosoma[i])
        intento.cromosoma = l
        intento.activos = [j for j in range(k_max) if intento.cromosoma[j] > 0.5]
        'existe la posibilidad, de que al cruzar no se cumpla que hayan almenos k_min cluster activos, si esto pasa reiniciamos los coeficientes'
        while len(intento.activos) < k_min:
          intento.cromosoma[:k_max] = np.random.random(k_max)
          lis = []
          for ts in range(k_max):
            if intento.cromosoma[ts] > 0.5:
              lis.append(ts)
          intento.activos = lis
        completar_individuo(intento, datos, k_max,f_distancia,x_min,x_max,aptitud,eps)
        if intento.aptitud > ind.aptitud:
          poblacion[poblacion.index(ind)]= intento
        if intento.aptitud > best.aptitud:
          best = intento
      if t in avance:
        print('vamos en la iteracion',t,'avance: ',t/10,'%')
    #print('mejor',best.cromosoma,'puntiacion',best.objetivo)
    mejores.append(best)
  imprimir(mejores,M)
  return(mejores)

In [ ]:
iris = pd.read_csv('/content/drive/MyDrive/csv/iris_data.csv',header=None);
iris.columns = [0,1,2,3,'clase']
iris.drop(['clase'],axis = 1,inplace=True)
iris = iris.to_numpy()

In [ ]:
glass = pd.read_csv('/content/drive/MyDrive/csv/glass_data.csv',header=None);
glass.columns = ['id',1,2,3,4,5,6,7,8,9,10]
glass.drop(['id'],axis = 1,inplace=True)
glass = glass.to_numpy()

In [ ]:
wine = pd.read_csv('/content/drive/MyDrive/csv/wine.csv')
wine.columns = ['id',0,1,2,3,4,5,6,7,8,9,10,11,12]
wine.drop(['id'],axis = 1,inplace=True)
wine = wine.to_numpy()

In [ ]:
#cancer = pd.read_csv('/content/drive/MyDrive/csv/breast-cancer-wisconsin.csv',header=None)

# Analizis cualitativo

## Indice de clustering: CS measure

### corriendo 30 veces el algoritmo

Iris

In [ ]:
ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.1,x_max=7.9,d = 4,
     max_eval=10000,datos=iris,f_distancia=euclideana,aptitud=cs,M = 30,eps=0.01)

DEspues de 30 iteraciones, estos son los resultados:
La mejor solucion fue [0.31847206145034423, 0.10546050463263262, 0.3978744962925649, 0.9739302510423562, 0.42944263547031547, 0.4492455980719591, 0.20328077371026054, 0.10854847349238805, 0.17081807551564288, 0.38792566199453365, 0.12791360010723987, 0.36634601020275037, 0.6687416309918903, 0.3073590288005701, 0.26694868494555224, 0.1908331962692586, 0.41983343167522613, 0.4274593908065854, 0.4927901923964413, 0.11473072131436168, [6.158626084468011, 3.0733251024945565, 4.043495058314957, 1.249402516923941], [7.720000000000001, 3.2, 6.660000000000001, 2.12], [1.7418902972890988, 1.895841002878568, 2.4693512810043874, 1.5608519110417833], [5.005999999999999, 3.4180000000000006, 1.464, 0.2439999999999999], [1.273940372709975, 3.057514167529745, 3.798728120753086, 6.203919638392355], [7.039273623755392, 5.091199088711615, 5.8452219095411095, 1.9572336146504496], [5.163293700416045, 3.336117903861824, 7.534900144942507, 4.913336807271491

Glass

In [ ]:
maximo = max([x for y in glass for x in y]);print(maximo)
minimo = min([x for y in glass for x in y]); print(minimo)

75.41
0.0


In [ ]:
ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.0,x_max=75.41,d = 10,
     max_eval=10000,datos=glass,f_distancia=euclideana,aptitud=cs,M = 30,eps=0.01)

DEspues de 30 iteraciones, estos son los resultados:
La mejor solucion fue [0.3367026228564235, 0.2684309035603588, 0.216265714423588, 0.19998282404455717, 0.31194270891553505, 0.04861871210539448, 0.21325943001106948, 0.3017638541902396, 0.09893528675266727, 0.9438450860203019, 0.40787973350564655, 0.3848739199928334, 1.0909251846117138, 0.4172819957475463, 0.09928603700801286, 0.4500564953958767, 0.02935540912617518, 0.08079471946704958, 0.4878764827435113, 0.23048267869053865, [1.5178418518518515, 13.51037037037037, 2.675925925925926, 1.465, 72.71388888888889, 0.529074074074074, 8.839629629629629, 0.0962962962962963, 0.05481481481481481, 2.7777777777777777], [41.129979418910374, 55.19710841078845, 46.03697735879916, 49.23563453803219, 56.712621852156474, 28.894591005167978, 2.5781717707879297, 63.27676803694739, 16.859426737457326, 28.84989205983568], [41.25037316330123, 64.62506387480212, 40.285005765790764, 57.828229735605284, 20.885781601977314, 17.59999599981295, 15.780202022118

Wine

In [ ]:
maximo = max([x for y in wine for x in y]);print(maximo)
minimo = min([x for y in wine for x in y]); print(minimo)

1680.0
0.13


In [ ]:
mej_wine = individuos = ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.13,x_max=1680,d = 13,
     max_eval=10000,datos=wine,f_distancia=euclideana,aptitud =cs,M = 30,eps=0.01)

DEspues de 30 iteraciones, estos son los resultados:
La mejor solucion fue [0.3807335715645147, 0.09188784075940315, 1.0563466122056688, 0.1654278700960401, 0.23287518983013766, 0.42487941239982163, 0.49918727164420673, 0.15952371498964898, 0.24412011031148007, 0.45332375435433914, 0.3081169127997194, 0.27367011758367854, 0.4799351583156205, 0.1300324127176159, 0.18254586310823256, 0.2409090862436443, 0.07254649182965212, 0.13348993168059642, 0.287377049352999, 0.7969012328397973, [594.7150481230593, 712.7250715888251, 898.2873317081733, 1473.5210253226924, 1578.7423649063296, 1558.864441246141, 684.0317535076738, 86.54242640819706, 698.7309078376287, 1334.2151221458762, 582.0354250397531, 1545.555360533119, 612.6045837504156], [365.2833339172953, 1049.620648135161, 960.7699051082868, 1524.2401351517312, 1199.4934407474732, 415.3745984039383, 1317.151882718158, 1019.1816938666269, 28.29267769558247, 170.1889399985774, 726.7701606265315, 1335.0687891363589, 221.89275072072476], [13.8278

### con diferentes epsilon

0.1

In [ ]:
mej_iris = ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.1,x_max=7.9,d = 4,
     max_eval=10000,datos=iris,f_distancia=euclideana,aptitud=cs,M = 30,eps=0.1)

DEspues de 30 iteraciones, estos son los resultados:
La mejor solucion fue [0.03535190900624274, 0.22540999261013323, 0.29327422581103485, 0.29627119759241527, 0.5852726977752174, 0.18352201039430271, 0.22523991138508473, 0.4943789338958451, 0.16269820312305813, 0.2507492055693883, 4.281896008276635, 0.34309442128813283, 0.12729708201578604, 0.26538451117269646, 0.13857132536247496, 0.35027306211306364, 0.15366487123126019, 0.40913542799436403, 0.2676205197939544, 0.23154394779190773, [5.467751670287348, 3.1828843132508773, 3.518307934014684, 1.1861927099295562], [7.358724531886491, 0.652064075250504, 2.135341467057674, 7.505983538720161], [6.095822091070751, 3.755718528306334, 7.445821221037168, 0.8307156834297224], [5.088992242522223, 7.178779564283835, 2.3617303220555437, 2.5733518310417267], [5.005999999999999, 3.4180000000000006, 1.464, 0.2439999999999999], [5.966516716330624, 3.154853335513092, 4.0397555718063405, 1.4084902236293244], [5.871264542248412, 6.529747549253519, 3.9581

In [ ]:
mej_glass= ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.0,x_max=75.41,d = 10,
     max_eval=10000,datos=glass,f_distancia=euclideana,aptitud=cs,M = 30,eps=0.1)

DEspues de 30 iteraciones, estos son los resultados:
La mejor solucion fue [0.47450356054846193, 0.3880144294895711, 0.04555696792442199, 0.05744815740891571, 0.36700358446298265, 0.3291394704300949, 0.3714537154704498, 0.37273053931773914, 0.07716716576541938, 0.06008535640365009, 0.34413970271249683, 1.179581004803871, 0.3848559709001499, 0.279357276543132, 0.38108437036180315, 0.21219362115424883, 0.44014403528181756, 0.055763277307733894, 0.22797390290925856, 9.233109059297616, [72.35587915799685, 72.06825877122334, 23.840960211287992, 57.05675411760465, 61.048662107200954, 75.32866061065067, 71.20679068619272, 45.00702187439121, 32.928557999365594, 58.09078393295098], [32.73406570521309, 13.881707822841056, 28.22477570151161, 66.69402003782005, 35.985122621753355, 29.83091357112503, 64.2279531008839, 47.828628889723895, 47.648456164201036, 1.257998658394639], [59.41743279188534, 67.70681436753252, 64.21020493334393, 20.17365668540024, 7.187005429297713, 32.182792389107874, 49.2218

0.001

In [ ]:
mej_iris = ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.1,x_max=7.9,d = 4,
     max_eval=10000,datos=iris,f_distancia=euclideana,aptitud=cs,M = 30,eps=0.001)

DEspues de 30 iteraciones, estos son los resultados:
La mejor solucion fue [0.46457025646882344, 0.9252394187467677, 0.34308896545388873, 0.39742036697056093, 0.006029021936098999, 0.3185445834362256, 0.07065042515375686, 0.038455563756702005, 0.27716520984006776, 0.22754967897149903, 0.2101911349283837, 0.45941345685530666, 0.5193298769043362, 0.10134909039054285, 0.4820692911837917, 0.4101313047981866, 0.12041956091622705, 0.05910675490415729, 0.33044562556736723, 0.4995586735201986, [5.4947777586499615, 3.1361443921171457, 3.4544066176044765, 1.1766156282533184], [5.005999999999999, 3.4180000000000006, 1.464, 0.2439999999999999], [6.705997615292801, 7.6133888542225545, 4.335295393482391, 7.554908972638874], [3.5281983835903246, 2.693254823129335, 3.943764802825633, 3.153212131974449], [4.9568830298693545, 2.2837682140258386, 1.658051828742148, 4.756826231141109], [6.412253725286111, 4.035135077660666, 1.1379182142845614, 3.400061695118786], [6.2978104414943665, 1.3816915946229784, 4

In [ ]:
mej_glass= ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.0,x_max=75.41,d = 10,
     max_eval=10000,datos=glass,f_distancia=euclideana,aptitud=cs,M = 30,eps=0.001)

DEspues de 30 iteraciones, estos son los resultados:
La mejor solucion fue [0.19467244665671019, 0.44540814245294363, 0.22530627815313786, 0.2903770297256957, 0.3832022229535853, 0.32605979512223704, 0.4406536441044877, 0.034180976396959606, 0.35784514845531556, 0.48141307931303495, 0.053573331314916706, 0.21027203473475614, 0.09460515637142752, 0.4836305371671236, 0.13457284160524985, 0.3489605640840199, 0.31433409264215406, 1.3371882304461502, 0.16522763463053103, 29.53549920999822, [61.58419771851992, 72.70629914574754, 3.432761877143224, 22.252092284374765, 25.486329204611202, 65.07537207998173, 36.03923986208191, 23.73138099942003, 20.738353110785372, 15.330160599579038], [52.80974391027315, 2.579095119364222, 69.0984754018172, 16.390728746185978, 49.76876827324543, 25.699422202508714, 18.27303625480344, 26.567856241053125, 69.72259794803736, 2.87218560115615], [57.63997905090902, 48.072839240082274, 30.92771621647818, 29.95710896243974, 51.800554235184926, 12.430692255433485, 8.1

In [ ]:
mej_wine = individuos = ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.13,x_max=1680,d = 13,
     max_eval=10000,datos=wine,f_distancia=euclideana,aptitud =DB,M = 30,eps=0.001)

In [ ]:
DB??

con distancia coseno

## Indice de clustering: DB Index

In [ ]:
ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.1,x_max=7.9,d = 4,
     max_eval=10000,datos=iris,f_distancia=euclideana,aptitud=cs,M = 30,eps=0.01)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
puntuaciones = []
for i in range(50):
  indK = individuo_vacio()
  kmeans = KMeans(n_clusters=3).fit(datos)
  indK.cromosoma = [1,1,1]
  for c in kmeans.cluster_centers_:
    indK.cromosoma.append(list(c))
  indK.particion = kmeans.labels_
  indK.activos = [0,1,2]
  puntuaciones.append(cs(indK,datos,3,euclideana))
np.mean(puntuaciones)

0.8186721431781523

In [ ]:
puntuaciones1 = []
for i in range(50):
  indK = individuo_vacio()
  kmeans = KMeans(n_clusters=9).fit(datos)
  indK.cromosoma = [1,1,1]
  for c in kmeans.cluster_centers_:
    indK.cromosoma.append(list(c))
  indK.particion = kmeans.labels_
  indK.activos = [0,1,2]
  puntuaciones1.append(cs(indK,datos,9,euclideana))
np.mean(puntuaciones1)

0.5671608002136336

In [ ]:
#version final funcional
def cs(individuo, datos : list,k_max,f_distancia):
  dividendo = []
  divisor = []
  centros = individuo.cromosoma[k_max::]
  for centro in individuo.activos:
    elementos = obtener_elementos(individuo.particion,centro,datos)
    centros[centro] = calcular_centroide(centro,elementos)
    'vamos a calcular la distancia mas grande de un punto a cualquier otro, para todos los puntos en el grupo'
    distancias_puntos = []
    for i in range(len( elementos)):
      distancias_puntos.append( max([f_distancia(elementos[i],elementos[x2],True) for x2 in range(len(elementos)) if x2 != i ] )    )
    dividendo.append(np.mean(distancias_puntos))
    'ahora calculamos las mimimas distancias entre los centroides de cada grupo'
    divisor.append( min([f_distancia(centros[centro],centros[x1]) for x1 in individuo.activos if x1 != centro] )  )
  'actualizamos la informacion del individuo, ya que alteramos una copia de sus datos'
  individuo.cromosoma[k_max::] = centros
  return(np.mean(dividendo) / np.mean(divisor))